In [9]:
import pandas as pd
import numpy as np
from numpy import tanh

In [10]:
# The sample
x = np.array([0.2, 0.3, 0.4])
y = 7.0

# Initialize Weights
Wc = np.array([0.2, 0.4])

Wo = np.array([0.1, 3.1])
Wf = np.array([2.3, 0.2])
Wi = np.array([3.1, 0.1])

Uc = np.array([[1.8, 3.6], [4.7, 2.9]])
Uo = np.array([[0.1, 0.9], [0.7, 4.3]])
Uf = np.array([[3.6, 4.1], [1.0, 0.9]])
Ui = np.array([[1.5, 2.6], [2.1, 0.2]])

w = np.array([2.0, 4.0])

# Backpropagation LSTM
$
% Forget-Gate
\newcommand{\F}{\color{purple}{f_t}}
\newcommand{\Ffull}[1][]{\color{purple}{\sigma^{#1} \left( W_f + U_f \cdot h_{t-1} \right)}}
\newcommand{\f}[1]{\color{purple}{f^t_{#1}}}
\newcommand{\ffull}[2][]{\color{purple}{\sigma^{#1} \left(w^f_{#2} x_t + u^f_{#2 1} h^{t-1}_1 + u^f_{#2 2}h^{t-1}_2 \right)}}
\newcommand{\Wf}{\color{purple}{W_f}}
\newcommand{\Uf}{\color{purple}{U_f}}
\newcommand{\wf}[1]{\color{purple}{w_{#1}^f}}
\newcommand{\uf}[2]{\color{purple}{u_{#1 #2}^f}}
% Input-Gate
\newcommand{\I}{\color{fuchsia}{i_t}}
\newcommand{\Ifull}[1][]{\color{fuchsia}{\sigma^{#1} \left( W_i + U_i \cdot h_{t-1} \right)}}
\newcommand{\i}[1]{\color{fuchsia}{i^t_{#1}}}
\newcommand{\ifull}[2][]{\color{fuchsia}{\sigma^{#1} \left( w^i_{#2} x_t + u^i_{#2 1} h^{t-1}_1 + u^i_{#2 2} h^{t-1}_2 \right)}}
\newcommand{\Wi}{\color{fuchsia}{W_i}}
\newcommand{\Ui}{\color{fuchsia}{U_i}}
\newcommand{\wi}[1]{\color{fuchsia}{w_{#1}^i}}
\newcommand{\ui}[2]{\color{fuchsia}{u_{#1 #2}^i}}
% Output-Gate
\newcommand{\O}{\color{turquoise}{o_t}}
\newcommand{\Ofull}[1][]{\color{turquoise}{\sigma^{#1} \left( W_o + U_o \cdot h_{t-1} \right)}}
\newcommand{\o}[1]{\color{turquoise}{o^t_{#1}}}
\newcommand{\ofull}[2][]{\color{turquoise}{\sigma^{#1} \left( w^o_{#2} x_t + u^o_{#2 1} h^{t-1}_1 + u^o_{#2 2} h^{t-1}_2 \right)}}
\newcommand{\Wo}{\color{turquoise}{W_o}}
\newcommand{\Uo}{\color{turquoise}{U_o}}
\newcommand{\wo}[1]{\color{turquoise}{w_{#1}^o}}
\newcommand{\uo}[2]{\color{turquoise}{u_{#1 #2}^o}}
% Cell (c without tilde - or however the fuck it's supposed to be called)
\newcommand{\C}[1][]{\color{lime}{c_{t #1}}}
\newcommand{\Cfull}{\F * \C[-1] + \I * \Ctil}
\newcommand{\c}[2][]{\color{lime}{c^{t #1}_{#2}}}
% Candidate (c with tilde - again: whatever, it's fine.)
\newcommand{\Ctil}{\color{green}{\tilde{c}_t}}
\newcommand{\Ctilfull}[1][]{\color{green}{g^{#1} \left( W_c x_t + U_c \cdot h_{t-1} \right)}}
\newcommand{\ctil}[2][]{\color{green}{g^{#1} \left( w^c_{#2} x_t + u^c_{#2 1} h^{t-1}_1 + u^c_{#2 2} h^{t-1}_2 \right)}}
\newcommand{\ctils}[1]{\color{green}{\tilde{c}_{#1}}}
\newcommand{\Wc}{\color{green}{W_c}}
\newcommand{\Uc}{\color{green}{U_c}}
\newcommand{\wc}[1]{\color{green}{w_{#1}^c}}
\newcommand{\uc}[2]{\color{green}{u_{#1 #2}^c}}
% Miscellaneous
\newcommand{\yhat}{\hat{y}_t}
\newcommand{\error}{\left(y - \yhat \right)}
\newcommand{\deriv}[2]{\frac{\partial #1}{\partial #2}}
\newcommand{\dxt}[1]{\color{#1}{x_t}}
\DeclareMathOperator{\diag}{diag}
\newcommand{\myH}[2][-1]{\color{#2}{h_{t #1}}}
\newcommand{\myh}[2][red]{\color{#1}{h^{t-1}_{#2}}}
$
In order to demonstrate how the backpropagation on the LSTM works we will further investigate our previous example. Remember, that we used a $3 \times 1$ vector for our single sample. Additionally, we looked at a very simple neural network that has only 2 neurons. Typically, we want to find better weights in each consecutive training iteration and in order to achieve that we follow a fraction of the negative gradient of a loss function.

So right now, we need two things:
1. a loss function
1. and its gradients with respect to each of the weight matrices.

Regarding the loss function we can make our lives easier by picking one, that is fit for our regression task while also being easy to derive. One of the easiest loss functions for that matter is the sum of squared errors, which we scale by a convenient constant:

$$loss_t = \frac{1}{2}\left( y - \yhat \right)^2.$$

Where $\yhat$ is the prediction at timestep $t$. When taking the derivative with respect to $\yhat$ one can easily see:

$$\deriv{loss_t}{\yhat} = \left( y - \yhat \right) (-1)$$

We define our total loss to be the sum of the losses at each timestep:

$$loss = \sum loss_t$$

This is also the reason, why the LSTM architecture remedies the vanishing or exploding gradients. Instead of resulting in a huge product as in the backpropagation through time, we will simply add the gradients of each timestep's contribution \[[Mallya 2017](http://arunmallya.github.io/writeups/nn/lstm/index.html)\].

Now that we know what function we want to derive we also need to know which weights we want to tune. We define the weight matrices as

\begin{alignat}{2}
% Weight matrices
\Wo &= \begin{bmatrix} \wo{1} \\ \wo{2} \end{bmatrix} \quad \Uo &&= \begin{bmatrix} \uo{1}{1} & \uo{1}{2} \\ \uo{2}{1} & \uo{2}{2} \end{bmatrix} \\
\Wf &= \begin{bmatrix} \wf{1} \\ \wf{2} \end{bmatrix} \quad \Uf &&= \begin{bmatrix} \uf{1}{1} & \uf{1}{2} \\ \uf{2}{1} & \uf{2}{2} \end{bmatrix} \\
\Wi &= \begin{bmatrix} \wi{1} \\ \wi{2} \end{bmatrix} \quad \Ui &&= \begin{bmatrix} \ui{1}{1} & \ui{1}{2} \\ \ui{2}{1} & \ui{2}{2} \end{bmatrix} \\
\Wc &= \begin{bmatrix} \wc{1} \\ \wc{2} \end{bmatrix} \quad \Uc &&= \begin{bmatrix} \uc{1}{1} & \uc{1}{2} \\ \uc{2}{1} & \uc{2}{2} \end{bmatrix}
\end{alignat}

Additionally, define the LSTM-forward pass as

\begin{align}
% Forward pass
\myH[]{red} &= \O * g(\C) \tag{1} \\
\C &= \F * \C[-1] + \I * \Ctil \tag{2} \\
\Ctil &= \Ctilfull = \begin{pmatrix} 
                         \ctil{1} \\ \ctil{2}
                     \end{pmatrix} \tag{3} \\
\I &= \begin{pmatrix} 
           \i{1} \\ \i{2} 
      \end{pmatrix} 
   = \Ifull
   = \begin{pmatrix}
         \ifull{1} \\ \ifull{2} 
     \end{pmatrix} \tag{4} \\
\F &= \begin{pmatrix} 
          \f{1} \\ \f{2} 
      \end{pmatrix} 
   = \Ffull 
   = \begin{pmatrix} 
         \ffull{1} \\ \ffull{2} 
     \end{pmatrix} \tag{5} \\
\O &= \begin{pmatrix} 
          \o{1} \\ \o{2} 
      \end{pmatrix} 
   = \Ofull
   = \begin{pmatrix} 
         \ofull{1} \\ \ofull{2}
     \end{pmatrix} \tag{6}
\end{align}

Where we choose the activation functions $\sigma$ as the sigmoid function and $g$ as the hyperbolic function \[[cp Dey, Salem 2017](https://arxiv.org/pdf/1701.05923.pdf)\].

\begin{alignat}{3}
\sigma(x) &= \frac{\exp(-x)}{1 + \exp(-x} &&\quad\text{with}\quad \deriv{\sigma(x)}{x} &&&= \sigma(x) (1-\sigma(x)) \\
g(x) &= \tanh(x) &&\quad\text{with}\quad \deriv{g(x)}{x} &&&= 1 - \tanh^2(x)
\end{alignat}

In [11]:
# Define activation functions
def sigmoid(x):
    """Sigmoid activation function."""
    return 1 / (1 + np.exp(-x))


def dsigmoid(x):
    """Derivative of sigmoid function."""
    return sigmoid(x) * (1 - sigmoid(x))


def dtanh(x):
    """Derivative of tanh function."""
    return 1 - np.tanh(x)**2

Let's also, for our purposes define the forward pass again, this time with a small change. This time, we want to store the results of every operation that depends on the timestep $t$ in their own lists. And we want to define a function that we can call later in a more convenient fashion.

In [12]:
# Redefine forward pass
h = []
o = []
f = []
i = []
c_ = []
c = []
y_ = []

def LSTM_forward():
    """Perform forward pass"""
    
    # First, set hidden state to zero
    h.append(np.zeros_like(Wc))
    c.append(np.zeros_like(Wc))
    
    # Set `start` so that future calls don't have to reset the above lists
    for t, xt in enumerate(x, start=len(y_)):
        # Calculate values of gates (equations 4, 5, 6)
        it = sigmoid(Wi.dot(xt) + Ui.dot(h[t]))
        ft = sigmoid(Wf.dot(xt) + Uf.dot(h[t]))
        ot = sigmoid(Wo.dot(xt) + Uo.dot(h[t]))

        # Calculate candidate update (equations 3 and 2)
        c_t = tanh(Wc.dot(xt) + Uc.dot(h[t]))
        ct = ft * c[t] + it * c_t

        # Calculate hidden state (equation 1)
        ht = ot * tanh(ct)

        # Prediction at step t (output layer)
        y_t = w.dot(ht)

        # Save variables to container, these will
        # persist outside of the function's scope
        h.append(ht)
        o.append(ot)
        f.append(ft)
        i.append(it)
        c_.append(c_t)
        c.append(ct)
        y_.append(y_t)
    
    # Return final prediction
    return y_[-1]

yhat_before = LSTM_forward()

Now, in order to derive the loss function, we need to figure out how we obtain our predictions. We know that the output layer is just a linear combination of the hidden state and a vector of weights which, in our case, we denote by $w \in \mathbb{R}^2$. So the next step is to take the neatly separated equations $(1)$ to $(6)$ and plug them all in.

\begin{equation}
\begin{split}
\yhat &= w^T \cdot \myH[]{red}
      = w^T \cdot \left[ \O * g(\C) \right] \\
      &= w^T \left[ \Ofull * g(\Cfull) \right] \\
      &= w^T \cdot \Bigg[ \begin{pmatrix} \ofull{1} \\ \ofull{2} \end{pmatrix} \\ 
      &\quad\quad* g 
                         \begin{pmatrix} 
                             \ffull{1} \c[-1]{1} + \ifull{1} + \ctil{1} \\ 
                             \ffull{2} \c[-1]{2} + \ifull{2} + \ctil{2}
                         \end{pmatrix} \Bigg]
\end{split}
\end{equation}

which, if written in its single equation form, gives

\begin{alignat}{2}
\yhat &= w_1 \ofull{1} g\left(\ffull{1} \c[-1]{1} + \\ \ifull{1} \ctil{1}\right)\\ 
      &+ w_2 \ofull{2} g\left(\ffull{2} \c[-1]{2} + \\ \ifull{2} \ctil{2}\right) \tag{7}
\end{alignat}

Now this is arguably neither neat, nor pretty. However, since we are looking at a very low dimensional example it is a good opportunity to look into the backpropagation algorithm using high-school math only. And we can do that in a straightforward manner looking at equation $(7)$.

Of course, if you are familiar with matrix derivatives and know that the element-wise multiplication of two vectors can be written as

\begin{equation}
a * b = \diag(a) \cdot b = \begin{bmatrix} 
                               a_1 & \dots & 0 \\
                               \vdots & \ddots & \vdots \\
                               0 & \dots & a_n
                           \end{bmatrix} \cdot
                           \begin{pmatrix} 
                                b_1 \\
                                \vdots \\
                                b_n
                           \end{pmatrix}
\end{equation}

then you're all set and you don't need to do this the tedious way we are about to embark on. Because we are going to take the partial derivative of $(7)$ with respect to each weight and using these results we will piece the gradients together. Moreover, once we have done that we will try to find a set of matrix operations, that will result in the gradients we previously puzzled together.

### Weights of the output gate

Let's start with the weights of the output gate. If we take the partial derivatives with respect to $\wo{1}$ and $\wo{2}$ respectively we obtain:

\begin{equation}
\deriv{\yhat}{\wo{1}} = w_1 g(\c{1}) \ofull[\prime]{1} \ctil{1} \dxt{turquoise}
\end{equation}

\begin{equation}
\deriv{\yhat}{\wo{2}} = w_2 g(\c{2}) \ofull[\prime]{2} \ctil{2} \dxt{turquoise}
\end{equation}

Which we now only need to combine in order to get the gradient of $\yhat$ with respect to $\Wo$.

\begin{align}
\deriv{\yhat}{\Wo} &= \begin{bmatrix} \deriv{\yhat}{\wo{1}} \\ \deriv{\yhat}{\wo{2}} \end{bmatrix} \\
                   &= w * g(\C) * \Ofull[\prime] * \Ctil\ \dxt{turquoise} \\
                   &= \diag(w) \cdot \diag(g(\C)) \cdot \diag(\Ofull[\prime]) \cdot \Ctil \ \dxt{turquoise}
\end{align}

The last line is just a way of describing the same formula, without the use of element-wise multiplication, relying just on scalar and dot products. However, while this satisfies more conventional algebraic notational standards it is computationally more efficient to implement the gradients using Hadamard products.

If we instead follow the logic of the backpropagation we obtain:

\begin{align}
\frac{\partial {loss}}{\partial W_o} &= \sum\frac{\partial {loss}_t}{\partial \hat{y}_t} \overbrace{\frac{\partial \hat{y}_t}{\partial h_t} \frac{\partial h_t}{\partial o_t} \frac{\partial o_t}{\partial \Wo}}^{\deriv{\yhat}{\Wo}}\\
                                   &= \sum -(y - \hat{y}_t)\ w * g(\C) * \Ofull[\prime] * \Ctil x_t.
\end{align}

Where the rightmost part is, unsurprisingly, the same as previously.

In [13]:
dLossdWo = np.zeros_like(Wo)

for t, xt in enumerate(x):
    # Note that c = (c0, c1, c2, c3), whereas y_ = (y_1, y_2, y_3)
    # thus we need to index c differently
    dLossdWo += -(y-y_[t]) * w *dsigmoid(Wo.dot(xt) + Uo.dot(h[t])) * tanh(c[t+1]) * c_[t] * xt

Similarly, we obtain the partial derivatives for each element of $\Uo$.

\begin{equation}
\deriv{\yhat}{\uo{1}{1}} = w_1 g(\c{1}) \ofull[\prime]{1} \ctil{1} \myh[turquoise]{1}
\end{equation}

\begin{equation}
\deriv{\yhat}{\uo{1}{2}} = w_1 g(\c{1}) \ofull[\prime]{1} \ctil{1} \myh[turquoise]{2}
\end{equation}

\begin{equation}
\deriv{\yhat}{\uo{2}{1}} = w_2 g(\c{2}) \ofull[\prime]{2} \ctil{2} \myh[turquoise]{1}
\end{equation}

\begin{equation}
\deriv{\yhat}{\uo{2}{2}} = w_2 g(\c{2}) \ofull[\prime]{2} \ctil{2} \myh[turquoise]{2}
\end{equation}

And by combining them into the jacobian of $\yhat$ with respect to $\Uo$ we get

\begin{align}
\deriv{\yhat}{\Uo} &= \begin{bmatrix}
                         \deriv{\yhat}{\uo{1}{1}} & \deriv{\yhat}{\uo{1}{2}} \\
                         \deriv{\yhat}{\uo{2}{1}} & \deriv{\yhat}{\uo{2}{2}}
                     \end{bmatrix} \\
                   &= \begin{bmatrix}
                         w_1 g(\c{1}) \ofull[\prime]{1} \ctils{1} \myh[turquoise]{1} &
                         w_1 g(\c{1}) \ofull[\prime]{1} \ctils{1} \myh[turquoise]{2} \\
                         w_2 g(\c{2}) \ofull[\prime]{2} \ctils{2} \myh[turquoise]{1} &
                         w_2 g(\c{2}) \ofull[\prime]{2} \ctils{2} \myh[turquoise]{2} \\
                     \end{bmatrix}  \\
                  &= \begin{bmatrix} w & w \end{bmatrix} *
                     \begin{bmatrix} g(\C) & g(\C) \end{bmatrix} *
                     \begin{bmatrix} \Ofull[\prime] & \Ofull[\prime] \end{bmatrix} *
                     \begin{bmatrix} \Ctil & \Ctil \end{bmatrix} *
                     \begin{bmatrix} \myH{turquoise}^T \\ \myH{turquoise}^T \end{bmatrix} 
                     \\
                  &= \diag(w) \cdot \diag(g(\C)) \cdot \diag(\Ofull[\prime]) \cdot \diag(\Ctil) \cdot \mathbb{1} \mathbb{1}^T \cdot \diag(\myH{turquoise}).
\end{align}

Or, by the logic of the backpropagation algorithm:

\begin{align}
\frac{\partial loss}{\partial U_o} &= \sum\frac{\partial {loss}_t}{\partial \hat{y}_t} \overbrace{\frac{\partial \hat{y}_t}{\partial h_t} \frac{\partial h_t}{\partial o_t} \frac{\partial o_t}{\partial \Uo}}^{\deriv{\yhat}{\Uo}} \\
                                   &= \sum -(y - \hat{y}_t)\ \begin{bmatrix} w & w \end{bmatrix} *
                     \begin{bmatrix} g(\C) & g(\C) \end{bmatrix} *
                     \begin{bmatrix} \Ofull[\prime] & \Ofull[\prime] \end{bmatrix} *
                     \begin{bmatrix} \Ctil & \Ctil \end{bmatrix} *
                     \begin{bmatrix} \myH{turquoise}^T \\ \myH{turquoise}^T \end{bmatrix}.
\end{align}

Again, rather than relying on the notation that makes heavy use of the dot products, we opt for implementing the one using element-wise multiplication. Note, that the first four of our matrix valued factors of that multiplication are obtained by combining copies of a vector in a column-wise fashion. In contrast to that, the last factor is obtained by row-wise stacking copies of a vector. In `numpy` one can obtain this behavior by a simple broadcasting operation. We simply need to `reshape` the first four factors, the expansion will then be performed automatically.

In [14]:
dLossdUo = np.zeros_like(Uo)

for t, xt in enumerate(x):
    dy_dot = w * tanh(c[t+1]) * dsigmoid(Wo.dot(xt) + Uo.dot(h[t])) * c_[t]
    # Expand so that the above multiplication can be performed element-wise
    dLossdUo += (y-y_[t]) * dy_dot.reshape(-1, 1) * h[t]

### Weights of the forget gate

By the same token we obtain the partial derivatives of $\yhat$ with respect to the elements of $\Wf$

\begin{equation}
\deriv{\yhat}{\wf{1}} = w_1 \o{1} g^\prime(\c{1}) \ffull[\prime]{1} \c[-1]{1} \dxt{purple}
\end{equation}

\begin{equation}
\deriv{\yhat}{\wf{2}} = w_2 \o{2} g^\prime(\c{2}) \ffull[\prime]{2} \c[-1]{2} \dxt{purple}
\end{equation}

and combine them to retrieve the gradient

\begin{align}
\deriv{\yhat}{\Wf} &= w * \O * g^\prime(\C) * \Ffull[\prime] * \C[-1] \ \dxt{purple} \\
                   &= \diag(w) \cdot \diag(\O) \cdot \diag\left(g^\prime(\C)\right) \cdot \diag(\Ffull[\prime]) \cdot \C[-1] \ \dxt{purple}.
\end{align}

Or, looking at the backpropagation's reasoning, we get:

\begin{align}
\frac{\partial loss}{\partial W_f} &= 
    \sum\frac{\partial loss_t}{\partial \hat{y}_t}\overbrace{\frac{\partial \hat{y}_t}{\partial h_t}\frac{\partial h_t}{\partial c_t} \frac{\partial c_t}{\partial f_t}\frac{\partial f_t}{\partial W_f}}^{\deriv{\yhat}{\Wf}} \\
                                   &= \sum -(y-\hat{y}_t) w * \left[\O * g^\prime(\C)\right] * \C * \Ffull[\prime] \dxt{purple}
\end{align}

Which we, straightforwardly, can implement.

In [15]:
dLossdWf = np.zeros_like(Wf)

for t, xt in enumerate(x):
    dLossdWf += -(y-y_[t]) * w * (o[t] * dtanh(c[t+1])) * c[t] * dsigmoid(Wf.dot(xt) + Uf.dot(h[t])) * xt

For the partial derivatives of $\yhat$ with respect to the elements of $\Uf$ we obtain:

\begin{equation}
\deriv{\yhat}{\uf{1}{1}} = w_1 g^\prime(\c{1}) \ffull[\prime]{1} \c[-1]{1} \myh[purple]{1}
\end{equation}

\begin{equation}
\deriv{\yhat}{\uf{1}{2}} = w_1 g^\prime(\c{1}) \ffull[\prime]{1} \c[-1]{1} \myh[purple]{2}
\end{equation}

\begin{equation}
\deriv{\yhat}{\uf{2}{1}} = w_2 g^\prime(\c{2}) \ffull[\prime]{2} \c[-1]{2} \myh[purple]{1}
\end{equation}

\begin{equation}
\deriv{\yhat}{\uf{2}{2}} = w_2 g^\prime(\c{2}) \ffull[\prime]{2} \c[-1]{2} \myh[purple]{2}.
\end{equation}

Which we can combine in order to acquire the jacobian of $\yhat$ with respect to $\Uf$.

\begin{align}
\deriv{\yhat}{\Uf} &= \begin{bmatrix}
                         \deriv{\yhat}{\uf{1}{1}} & \deriv{\yhat}{\uf{1}{2}} \\
                         \deriv{\yhat}{\uf{2}{1}} & \deriv{\yhat}{\uf{2}{2}}
                     \end{bmatrix}
                   = \begin{bmatrix}
                          w_1 g^\prime(\c{1}) \ffull[\prime]{1} \c[-1]{1} \myh[purple]{1} &
                          w_1 g^\prime(\c{1}) \ffull[\prime]{1} \c[-1]{1} \myh[purple]{2} \\
                          w_2 g^\prime(\c{2}) \ffull[\prime]{2} \c[-1]{2} \myh[purple]{1} &
                          w_2 g^\prime(\c{2}) \ffull[\prime]{2} \c[-1]{2} \myh[purple]{2}
                     \end{bmatrix}
                  \\
                  &= \begin{bmatrix} w & w \end{bmatrix} *
                     \begin{bmatrix} \O & \O \end{bmatrix} *
                     g^\prime( \begin{bmatrix} \C & \C \end{bmatrix}) *
                     \begin{bmatrix} \Ffull[\prime] & \Ffull[\prime] \end{bmatrix} *
                     \begin{bmatrix} \C[-1] & \C[-1] \end{bmatrix} *
                     \begin{bmatrix} \myH{purple}^T \\ \myH{purple}^T \end{bmatrix} \\
                  &= \diag(w) \cdot \diag(\O) \cdot \diag\left(g^\prime(\C)\right) \cdot \diag(\Ffull[\prime]) \cdot \diag(\C[-1]) \cdot \mathbb{1} \mathbb{1}^T \cdot \diag(\myH{purple})
\end{align}

The same is of course achieved by the backpropagation's rationale.

\begin{align}
\frac{\partial loss}{\partial U_f} &= \sum\frac{\partial loss_t}{\partial \hat{y}_t} \overbrace{\frac{\partial \hat{y}_t}{\partial h_t} \frac{\partial h_t}{\partial c_t} \frac{\partial c_t}{\partial f_t} \frac{\partial f_t}{\partial U_f}}^{\deriv{\yhat}{\Uf}} \\
                                   &= \sum -(y-\hat{y}_t)\ \begin{bmatrix} w & w \end{bmatrix} *
                     \begin{bmatrix} \O & \O \end{bmatrix} *
                     g^\prime( \begin{bmatrix} \C & \C \end{bmatrix}) *
                     \begin{bmatrix} \Ffull[\prime] & \Ffull[\prime] \end{bmatrix} *
                     \begin{bmatrix} \C[-1] & \C[-1] \end{bmatrix} *
                     \begin{bmatrix} \myH{purple}^T \\ \myH{purple}^T \end{bmatrix}
\end{align}

Which we can implement, making use of `numpy`'s broadcasting operations, just as before.

In [16]:
dLossdUf = np.zeros_like(Uf)

for t, xt in enumerate(x):
    dy_dft = w*(o[t]*dtanh(c[t+1])) * c[t] * dsigmoid(Wf.dot(xt)+Uf.dot(h[t]))
    dLossdUf += -(y-y_[t]) * dy_dft.reshape(-1, 1) * h[t]

### Weights of the input gate

Looking at the weights of the input gate we can proceed in the familiar fashion. We procure the partial derivatives of $\yhat$ with respect to $\wi{1}$ and $\wi{2}$

\begin{equation}
\deriv{\yhat}{\wi{1}} = w_1 \o{1} g^\prime(\c{1}) \ctil{1} \ifull[\prime]{1} \dxt{fuchsia},
\end{equation}


\begin{equation}
\deriv{\yhat}{\wi{2}} = w_2 \o{2} g^\prime(\c{2}) \ctil{2} \ifull[\prime]{2} \dxt{fuchsia}
\end{equation}

and proceed by collecting them in order to form the gradient of $\yhat$ with respect to $\Wi$

\begin{align}
\deriv{\yhat}{\Ui} &= w * \O * g^\prime(\C) * \Ctil * \Ifull[\prime] \dxt{fuchsia} \\
                   &= \diag(w) \cdot \diag(\O) \cdot \diag(g^\prime(\C)) \cdot \diag(\Ctil) \cdot \Ifull[\prime] \dxt{fuchsia}.
\end{align}

Again, looking at it from the backpropagation's view, we get

\begin{align}
\frac{\partial loss}{\partial W_i} &= \sum\frac{\partial loss_t}{\partial \hat{y}_t} \overbrace{\frac{\partial \hat{y}_t}{\partial h_t}\frac{\partial h_t}{\partial c_t} \frac{\partial c_t}{\partial i_t}\frac{\partial i_t}{\partial \Wi}}^{\deriv{\yhat}{\Wi}} \\
                                   &= \sum -(y-\hat{y}_t)\ w * \O * g^\prime(\C) * \Ctil * \Ifull[\prime] \dxt{fuchsia}.
\end{align}

Subsequently, we can implement the computation of the gradient.

In [17]:
dLossdWi = np.zeros_like(Wi)

for t, xt in enumerate(x):
    dLossdWi += -(y-y_[t]) * w * (o[t] * dtanh(c[t+1])) * c_[t] * dsigmoid(Wi.dot(xt) + Ui.dot(h[t])) * xt

Equivalently we proceed with the partial derivatives of $\yhat$ with respect to the elements of $\Ui$, which are

\begin{equation}
\deriv{\yhat}{\ui{1}{1}} = w_1 \o{1} g^\prime(\c{1}) \ctil{1} \ifull[\prime]{1} \myh[fuchsia]{1},
\end{equation}

\begin{equation}
\deriv{\yhat}{\ui{1}{2}} = w_1 \o{1} g^\prime(\c{1}) \ctil{1} \ifull[\prime]{1} \myh[fuchsia]{2},
\end{equation}

\begin{equation}
\deriv{\yhat}{\ui{2}{1}} = w_2 \o{2} g^\prime(\c{2}) \ctil{2} \ifull[\prime]{2} \myh[fuchsia]{1},
\end{equation}
and
\begin{equation}
\deriv{\yhat}{\ui{2}{2}} = w_2 \o{2} g^\prime(\c{2}) \ctil{2} \ifull[\prime]{2} \myh[fuchsia]{2}.
\end{equation}

By collecting them into a matrix we obtain the jacobian of $\yhat$ with respect to $\Ui$

\begin{align}
\deriv{\yhat}{\Ui} &= \begin{bmatrix}
                          \deriv{\yhat}{\ui{1}{1}} & \deriv{\yhat}{\ui{1}{2}} \\
                          \deriv{\yhat}{\ui{2}{1}} & \deriv{\yhat}{\ui{2}{2}} 
                      \end{bmatrix} \\ 
                   &= \begin{bmatrix}
                          w_1 \o{1} g^\prime(\c{1}) \ctils{1} \ifull[\prime]{1} \myh[fuchsia]{1} &
                          w_1 \o{1} g^\prime(\c{1}) \ctils{1} \ifull[\prime]{1} \myh[fuchsia]{2} \\
                          w_2 \o{2} g^\prime(\c{2}) \ctils{2} \ifull[\prime]{2} \myh[fuchsia]{1} &
                          w_2 \o{2} g^\prime(\c{2}) \ctils{2} \ifull[\prime]{2} \myh[fuchsia]{2} \\
                      \end{bmatrix} \\
                   &= \begin{bmatrix} w & w \end{bmatrix} *
                      \begin{bmatrix} \O & \O \end{bmatrix} *
                      g^\prime\left(\begin{bmatrix} \C & \C \end{bmatrix}\right) *
                      \begin{bmatrix} \Ctil & \Ctil \end{bmatrix} *
                      \begin{bmatrix} \Ifull[\prime] & \Ifull[\prime] \end{bmatrix} *
                      \begin{bmatrix} \myH{fuchsia}^T \\ \myH{fuchsia}^T \end{bmatrix} \\
                   &= \diag(w) \cdot \diag(\O) \cdot \diag\left(g^\prime(\C)\right) \cdot \diag(\Ctil) \cdot \diag(\Ifull[\prime]) \cdot \mathbb{1} \mathbb{1}^T \cdot \diag(\myH{fuchsia})
\end{align}

For our intents and purposes we are interested in the gradients of the loss function, which are

\begin{align}
\frac{\partial loss}{\partial U_i} &= \sum\frac{\partial loss_t}{\partial \hat{y}_t} \overbrace{\frac{\partial \hat{y}_t}{\partial h_t} \frac{\partial h_t}{\partial c_t} \frac{\partial c_t}{\partial i_t} \frac{\partial i_t}{\partial U_i}}^{\deriv{\yhat}{\Ui}} \\
                                   &= \sum(y-\hat{y}_t)\ \begin{bmatrix} w & w \end{bmatrix} *
                      \begin{bmatrix} \O & \O \end{bmatrix} *
                      g^\prime\left(\begin{bmatrix} \C & \C \end{bmatrix}\right) *
                      \begin{bmatrix} \Ctil & \Ctil \end{bmatrix} * \\
                      &\quad\quad\begin{bmatrix} \Ifull[\prime] & \Ifull[\prime] \end{bmatrix} *
                      \begin{bmatrix} \myH{fuchsia}^T \\ \myH{fuchsia}^T \end{bmatrix}
\end{align}

Which we implement by making use of `numpy`'s broadcasting functionality.

In [18]:
dLossdUi = np.zeros_like(Ui)

for t, xt in enumerate(x):
    dy_dit = w * (o[t]*dtanh(c[t])) * c_[t] * dsigmoid(Wi.dot(xt)+Ui.dot(h[t]))
    dLossdUi += -(y-y_[t]) * dy_dit.reshape(-1, 1) * h[t]

### Weights of the candidate update

Again we begin by taking partial derivatives of $\yhat$ with respect to $\Wc$'s elements, such that we get

\begin{align}
\deriv{\yhat}{\wc{1}} = w_1 \o{1} g^\prime(\c{1}) \ifull{1} \ctil[\prime]{1} \dxt{green}
\end{align}

and

\begin{align}
\deriv{\yhat}{\wc{2}} = w_2 \o{2} g^\prime(\c{2}) \ifull{2} \ctil[\prime]{2} \dxt{green}.
\end{align}

Arranging the partial derivatives, we obtain the gradient of $\yhat$ with respect to $\Wc$

\begin{align}
\deriv{\yhat}{\Wc} = w * \O * g^\prime(\C) * \I * \Ctilfull[\prime]\ \dxt{green},
\end{align}

which, in terms of our loss function gives


\begin{align}
\frac{\partial loss}{\partial W_c} &= \sum\frac{\partial loss_t}{\partial \hat{y}_t}\overbrace{\frac{\partial \hat{y}_t}{\partial h_t}\frac{\partial h_t}{\partial c_t}\frac{\partial c_t}{\partial \tilde{c}_t} \frac{\partial \tilde{c}_t}{\partial W_c}}^{\deriv{\yhat}{\Wc}} \\
                                   &= \sum(y - \hat{y}_t)\ w * \O * g^\prime(\C) * \I * \Ctilfull[\prime]\ \dxt{green}
\end{align}

which we implement in the same manner as before.

In [19]:
dLossdWc = np.zeros_like(Wc)

for t, xt in enumerate(x):
    dLossdWc += -(y-y_[t]) * w * (o[t] * dtanh(c[t+1])) * i[t] * dtanh(Wc.dot(xt) + Uc.dot(h[t])) * xt

Similarly, we start by obtaining the partial derivatives of $\yhat$ with respect to each of $\Wc$'s elements.

\begin{equation}
\deriv{\yhat}{\uc{1}{1}} = w_1 \o{1} g^\prime(\c{1}) \i{1} \ctil[\prime]{1} \myh[green]{1},
\end{equation}

\begin{equation}
\deriv{\yhat}{\uc{1}{2}} = w_1 \o{1} g^\prime(\c{1}) \i{1} \ctil[\prime]{1} \myh[green]{2},
\end{equation}

\begin{equation}
\deriv{\yhat}{\uc{2}{1}} = w_2 \o{2} g^\prime(\c{2}) \i{2} \ctil[\prime]{2} \myh[green]{1}
\end{equation}
and
\begin{equation}
\deriv{\yhat}{\uc{2}{2}} = w_2 \o{2} g^\prime(\c{2}) \i{2} \ctil[\prime]{2} \myh[green]{2}.
\end{equation}

And by organizing these partial derivatives, we can form the jacobian of $\yhat$ with respect to $\Uc$

\begin{align}
\deriv{\yhat}{\Uc} &= \begin{bmatrix} 
                          \deriv{\yhat}{\uc{1}{1}} & \deriv{\yhat}{\uc{1}{2}} \\
                          \deriv{\yhat}{\uc{2}{1}} & \deriv{\yhat}{\uc{2}{2}}
                      \end{bmatrix} = 
                      \begin{bmatrix}
                          w_1 \o{1} g^\prime(\c{1}) \i{1} \ctil[\prime]{1} \myh[green]{1} &
                          w_1 \o{1} g^\prime(\c{1}) \i{1} \ctil[\prime]{1} \myh[green]{2} \\
                          w_2 \o{2} g^\prime(\c{2}) \i{2} \ctil[\prime]{2} \myh[green]{1} &
                          w_2 \o{2} g^\prime(\c{2}) \i{2} \ctil[\prime]{2} \myh[green]{2} 
                      \end{bmatrix} \\
                   &= \begin{bmatrix} w & w \end{bmatrix} *
                      \begin{bmatrix} \O & \O \end{bmatrix} *
                      g^\prime(\begin{bmatrix} \C & \C \end{bmatrix}) *
                      \begin{bmatrix} \I & \I \end{bmatrix} *
                      \begin{bmatrix} \Ctilfull[\prime] & \Ctilfull[\prime] \end{bmatrix} *
                      \begin{bmatrix} \myH{green}^T \\ \myH{green}^T \end{bmatrix} \\
                   &= \diag(w) \cdot 
                      \diag(\O) \cdot 
                      \diag\left(g^\prime(\C)\right) \cdot 
                      \diag(\I) \cdot 
                      \diag(\Ctilfull[\prime]) \cdot 
                      \mathbb{1} \mathbb{1}^T \cdot 
                      \diag(\myH{green})
\end{align}

For our problem this means, that we calculate the gradient of the loss function with respect to $\Uc$ as

\begin{align}
\frac{\partial loss}{\partial U_c} &= \sum\frac{\partial loss_t}{\partial \hat{y}_t}\overbrace{\frac{\partial \hat{y}_t}{\partial h_t}\frac{\partial h_t}{\partial c_t}\frac{\partial c_t}{\partial \tilde{c}_t}\frac{\partial \tilde{c}_t}{\partial U_c}}^{\deriv{\yhat}{\Uc}} \\
                          &= \sum(y_t - \hat{y}_t)\ w * \O * g^\prime(\C) * \I * \Ctilfull[\prime]\ * \begin{bmatrix} h^T_{t-1} \\ h^T_{t-1} \end{bmatrix},
\end{align}

which by now should be easy to implement.

In [20]:
dLossdUc = np.zeros_like(Uc)

for t, xt in enumerate(x):
    dc_tdf = w * (o[t] * dtanh(c[t+1])) * i[t] * dtanh(Wc.dot(xt)+Uc.dot(h[t]))
    dLossdUc += -(y-y_[t]) * dc_tdf.reshape(-1, 1) * h[t]

### Weights of the output layer

Finally, we need to adjust the weights of the output layer. The gradient of the loss function is simply

\begin{equation}
\deriv{loss}{w} = \sum\deriv{loss_t}{w} = \sum\deriv{loss_t}{\yhat}\deriv{\yhat}{w} = \sum -(y - \yhat) \myH[]{red}.
\end{equation}

Which can be readily implemented.

In [21]:
dLossdw = np.zeros_like(w)

for t, xt in enumerate(x):
    dLossdw += -(y - y_[t]) * h[t+1]

## Weight update

With all gradients being computed, we can use them in order to update our weights. Typically, when using gradient descent, we follow the direction of the negative gradient, multiplied by some fraction.

In [22]:
eta = 0.01

Wc -= eta * dLossdWc
Wo -= eta * dLossdWo
Wf -= eta * dLossdWf
Wi -= eta * dLossdWi

Uc -= eta * dLossdUc
Uo -= eta * dLossdUo
Uf -= eta * dLossdUf
Ui -= eta * dLossdUi

w -= eta * dLossdw

With the new weights we can compute the updated prediction.

In [23]:
yhat_after = LSTM_forward()

In [24]:
pd.DataFrame({r"$\hat{y}_{before}$": yhat_before, r"$\hat{y}_{after}$": yhat_after, r"$y$": y}, index=[1])

,$\hat{y}_{before}$,$\hat{y}_{after}$,$y$
1,2.046038,4.956741,7.0


As we can clearly see, we are getting close to the real $y$. In a real application one would of course evaluate the gradients at multiple samples and use an average gradient in order to update the weight matrices.